In [1]:
import cv2
import json
import os
import mediapipe as mp
import pandas as pd
import numpy as np

In [9]:
from tqdm import tqdm

In [2]:
directory = 'D:\\ML\\Attention-focus-detection-daniyar\\data\\maxim\\'

In [3]:
mypath = 'D:\\ML\\Attention-focus-detection-daniyar\\data\\maxim\\json nt.json'
myjson = open(mypath, "r").read()
json_details = json.loads(myjson)
#myjson.close()

In [27]:
#json_details['_via_img_metadata']['105524417.jpeg1111088']['regions'][0]['shape_attributes']['x']
#list(json_details['_via_img_metadata'].keys())

In [28]:
#json_details['_via_img_metadata']['105524417.jpeg1111088']['regions'][0]['shape_attributes']['x']

In [1]:
dict_faces = {}
#Создается пустой словарь, который будет содержать данные о лицах на изображениях.
for f in list(json_details['_via_img_metadata'].values()):
    temp_list = []
    for region in f['regions']:      
        temp_list.append({'x':int(region['shape_attributes']['x']),
                                    'y':int(region['shape_attributes']['y']),
                                    'width':int(region['shape_attributes']['width']),
                                    'height':int(region['shape_attributes']['height'])
                                    })
    dict_faces[f['filename']] = temp_list
dict_faces

NameError: name 'json_details' is not defined

In [5]:
list(json_details['_via_img_metadata'].keys())

['5a394610b890d909156903.jpg725848',
 'scale_1200.jpeg238970',
 'e863687674f78533a62fd51266d3be90.jpg325006',
 '1597801.jpg865733',
 'ria_2712484hr_63c.jpg1347523',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22.jpg199592',
 '44218203881_968f50a0ff_k.jpg954326',
 '1-1-1.jpg234474',
 '28_n1976129_big.jpg439807',
 'scale_1200 (1).png969032',
 'G8CiTZAxMMg.jpg210209',
 '6307797c6e48fe5cb01f2b3d7726f839.jpeg582293',
 'Nuzhno-li-xodit-na-lekcii.jpg98631',
 'img_3475_2b_2.jpg303401',
 '1559569135_8N5B8535.jpg1595991',
 'С…С…С….jpg1965171',
 '4yg895CH6II.jpg182228',
 '1673735382_pro-dachnikov-com-p-foto-za-partoi-v-univere-4.jpg140537',
 '1673971580_pro-dachnikov-com-p-studenti-za-partami-foto-60.jpg240110',
 'foto._2.iz.6_.jpg110430',
 '6389af16039c6663274986.jpg306704',
 'photo_002.jpg232856',
 'studenty-na-lekczii-1-scaled.jpg424346',
 '1683640998_kartinkof-club-p-kartinki-lektsii-37.jpg2483632',
 'e407d64acb0ec1530efb97fb6b8d3b19.jpg158097',
 '2002461.jpg132211',
 'Zn0R0T54q

In [8]:
"""for file in os.listdir(directory):
    if file.split('.')[-1] == 'jpg':
        st_photo = cv2.imread(directory + '\\' + file, 1)
        #dict_faces[file]['x']
        for face_rectangle in dict_faces[file]:
            cv2.rectangle(st_photo, 
                        (face_rectangle['x'], face_rectangle['y']), 
                        (face_rectangle['x'] + face_rectangle['width'], face_rectangle['y'] + face_rectangle['height']),
                        (0, 0, 255), 
                        3)
        cv2.imshow(file, st_photo)
        cv2.waitKey(0)
        cv2.destroyAllWindows()"""

"for file in os.listdir(directory):\n    if file.split('.')[-1] == 'jpg':\n        st_photo = cv2.imread(directory + '\\' + file, 1)\n        #dict_faces[file]['x']\n        for face_rectangle in dict_faces[file]:\n            cv2.rectangle(st_photo, \n                        (face_rectangle['x'], face_rectangle['y']), \n                        (face_rectangle['x'] + face_rectangle['width'], face_rectangle['y'] + face_rectangle['height']),\n                        (0, 0, 255), \n                        3)\n        cv2.imshow(file, st_photo)\n        cv2.waitKey(0)\n        cv2.destroyAllWindows()"

## КОД ДЛЯ РАЗМЕТКИ "есть фокус/нет фокуса"

In [7]:
file_counter = 0
for file in os.listdir(directory):
    if file.split('.')[-1] == 'jpg':
        st_photo = cv2.imread(directory + '\\' + file, 1)
        #dict_faces[file]['x']
        file_counter += 1
        face_counter = 0
        for face_rectangle in dict_faces[file]:
            #(face_rectangle['x'], face_rectangle['y']), 
            #face_rectangle['x'] + face_rectangle['width'], face_rectangle['y'] + face_rectangle['height'])
            st_face = st_photo[face_rectangle['y']:face_rectangle['y'] + face_rectangle['height'], face_rectangle['x']:face_rectangle['x'] + face_rectangle['width'],:]
            cv2.imshow(file + ' ' + str(face_counter), st_face)
            face_counter += 1
            key = cv2.waitKey(0)
            if key == 48:
                path_0 = 'D:\\ML\\Attention-focus-detection\\data\\0\\'
                cv2.imwrite('D:\\ML\\Attention-focus-detection\\data\\0\\' + file.split('.')[0] + '_' + str(face_counter) + '.jpg', st_face)
            elif key == 49:
                path_1 = 'D:\\ML\\Attention-focus-detection\\data\\1\\'
                cv2.imwrite('D:\\ML\\Attention-focus-detection\\data\\1\\' + file.split('.')[0] + '_' + str(face_counter) + '.jpg', st_face)
            cv2.destroyAllWindows()
        #if file_counter >= 2:
        #    break

KeyError: 'ххх.jpg'

## КОД для обработки через mediapipe

In [3]:
path_0 = 'D:\\ML\\Attention-focus-detection-daniyar\\data\\0\\'
path_1 = 'D:\\ML\\Attention-focus-detection-daniyar\\data\\1\\'

In [5]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) 

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [7]:
[f for f in os.listdir(path_0) if os.path.isfile(os.path.join(path_0,f))]

['1-1-1_1.jpg',
 '1-1-1_2.jpg',
 '1-1-1_3.jpg',
 '1-1-1_4.jpg',
 '1-1-1_5.jpg',
 '1-1-1_6.jpg',
 '1-1-1_7.jpg',
 '1-1-1_8.jpg',
 '1-1-1_9.jpg',
 '1559569135_8N5B8535_2.jpg',
 '1559569135_8N5B8535_3.jpg',
 '1597801_1.jpg',
 '1597801_10.jpg',
 '1597801_11.jpg',
 '1597801_14.jpg',
 '1597801_15.jpg',
 '1597801_16.jpg',
 '1597801_17.jpg',
 '1597801_2.jpg',
 '1597801_20.jpg',
 '1597801_21.jpg',
 '1597801_22.jpg',
 '1597801_23.jpg',
 '1597801_24.jpg',
 '1597801_25.jpg',
 '1597801_3.jpg',
 '1597801_5.jpg',
 '1597801_6.jpg',
 '1597801_8.jpg',
 '1597801_9.jpg',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22_1.jpg',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22_2.jpg',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22_3.jpg',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22_4.jpg',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22_5.jpg',
 '1673735365_pro-dachnikov-com-p-foto-za-partoi-v-univere-22_6.jpg',
 '1673735365_pro-dachnikov-c

## КОД для создания обучающего датасета

In [8]:
dataset_path = 'dataset.csv'
df = pd.DataFrame(columns=np.arange(478 * 3 + 2))

for file in [f for f in os.listdir(path_0) if os.path.isfile(os.path.join(path_0,f))]:
    print(path_0 + file)
    image = cv2.imread(path_0 + file, 1)

    image.flags.writeable = False
    image.flags.writeable = True

    results = face_mesh.process(image)
    #if not results.multi_face_landmarks:
    #    print(file)
    #    continue
    annotated_image = image.copy()
    try:
        for face_landmarks in results.multi_face_landmarks:

            lm_list = [file, 0]
            #print('face_landmarks:', (face_landmarks))
            #print(len(face_landmarks.landmark))
            for lm in face_landmarks.landmark:
                lm_list.append(lm.x)
                lm_list.append(lm.y)
                lm_list.append(lm.z)

            df.loc[len(df)] = lm_list

            #print(face_landmarks[0])
            mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
    except TypeError as exc:
        print(exc)

    #cv2.imwrite(path_0 + '//mp//' + str(file) + '.png', annotated_image)
df.to_csv('dataset.csv', sep=';')

D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_1.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_2.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_3.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_4.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_5.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_6.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_7.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_8.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1-1-1_9.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1559569135_8N5B8535_2.jpg
'NoneType' object is not iterable
D:\ML\Attention-focus-detection-daniyar\data\0\1559569135_8N5B8535_3.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1597801_1.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1597801_10.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1597801_11.jpg
D:\ML\Attention-focus-detection-daniyar\data\0\1597801_14.jpg
D:\ML\Attention-focus-detection-daniyar\da

In [10]:
dataset_path = 'dataset.csv'

for file in tqdm([f for f in os.listdir(path_1) if os.path.isfile(os.path.join(path_1,f))]):
    print(path_1 + file)
    image = cv2.imread(path_1 + file, 1)

    image.flags.writeable = False
    image.flags.writeable = True

    results = face_mesh.process(image)
    

    if not results.multi_face_landmarks:
        print(file)
        continue

    annotated_image = image.copy()
    try:
        for face_landmarks in results.multi_face_landmarks:

            lm_list = [file, 1]
            #print('face_landmarks:', (face_landmarks))
            #print(len(face_landmarks.landmark))
            for lm in face_landmarks.landmark:
                lm_list.append(lm.x)
                lm_list.append(lm.y)
                lm_list.append(lm.z)

            df.loc[len(df)] = lm_list

            #print(face_landmarks[0])
            mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
            
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
    except TypeError as exc:
        print(exc)
        
    #cv2.imwrite(path_0 + '//mp//' + str(file) + '.png', annotated_image)
df.to_csv('dataset.csv', sep=';')

  0%|          | 0/225 [00:00<?, ?it/s]

D:\ML\Attention-focus-detection-daniyar\data\1\1559569135_8N5B8535_1.jpg


  1%|▏         | 3/225 [00:00<00:27,  7.95it/s]

D:\ML\Attention-focus-detection-daniyar\data\1\1559569135_8N5B8535_4.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\1597801_12.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\1597801_13.jpg


  2%|▏         | 5/225 [00:00<00:22,  9.97it/s]

...

D:\ML\Attention-focus-detection-daniyar\data\1\st93_1.jpg
st93_1.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\st93_2.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\st94_1.jpg
st94_1.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\st94_2.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\st95_1.jpg


 98%|█████████▊| 221/225 [00:18<00:00, 13.87it/s]

D:\ML\Attention-focus-detection-daniyar\data\1\st96_1.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\st97_1.jpg


100%|██████████| 225/225 [00:19<00:00, 11.72it/s]


D:\ML\Attention-focus-detection-daniyar\data\1\st99_1.jpg
D:\ML\Attention-focus-detection-daniyar\data\1\studenty-na-lekczii-1-scaled_1.jpg
